### Packages

In [5]:
!pip install mord
!pip install dmba

In [6]:
import warnings
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor, NearestNeighbors
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from dmba import gainsChart, liftChart
from dmba import classificationSummary
from sklearn.model_selection import cross_val_score


import statsmodels.api as sm
from mord import LogisticIT

no display found. Using non-interactive Agg backend


### Data Loading and cleaning

In [8]:
crime_df = pd.read_csv('Crimes_-_2001_to_Present.csv')

In [10]:
crime_df_no_na = crime_df.dropna()

In [11]:
crime_df_drop = crime_df_no_na.drop(columns=[
    'ID', 'Case Number','Block', 'IUCR', 'FBI Code', 'X Coordinate', 'Y Coordinate',
    'Updated On', 'Location', 'Location Description', 'Latitude', 'Longitude',
    'Year'])

In [12]:
crime_encoded = pd.get_dummies(crime_df_drop, columns= ['Primary Type', 'Description'], drop_first=True)

In [13]:
crime_sample_df = crime_encoded.sample(frac=0.01, random_state=7)

In [14]:
crime_sample_df.head()

,Date,Arrest,Domestic,Beat,District,Ward,Community Area,Primary Type_ASSAULT,Primary Type_BATTERY,Primary Type_BURGLARY,...,Description_VIOLATION OF SUMMARY CLOSURE,Description_VIOLENT OFFENDER - ANNUAL REGISTRATION,Description_VIOLENT OFFENDER - DUTY TO REGISTER,Description_VIOLENT OFFENDER - FAIL TO REGISTER NEW ADDRESS,Description_VIOLENT OFFENDER: ANNUAL REGISTRATION,Description_VIOLENT OFFENDER: DUTY TO REGISTER,Description_VIOLENT OFFENDER: FAIL TO REGISTER NEW ADDRESS,Description_WIC FRAUD,Description_WIREROOM/HORSES,Description_WIREROOM/SPORTS
2721891,08/21/2006 02:20:00 AM,True,False,313,3.0,20.0,42.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2461538,11/23/2005 05:05:00 PM,False,True,723,7.0,17.0,68.0,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4695736,04/22/2011 05:32:00 PM,False,True,2233,22.0,34.0,49.0,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1984789,11/26/2004 12:20:00 AM,False,True,1531,15.0,37.0,25.0,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2621547,03/24/2006 07:30:00 PM,False,False,733,7.0,17.0,68.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [15]:
crime_sample_df['Date'] = pd.to_datetime(crime_sample_df['Date'])
crime_sample_df['hour'] = crime_sample_df['Date'].dt.hour
crime_sample_df['day_of_week'] = crime_sample_df['Date'].dt.dayofweek

### KNN

In [17]:
crime_sample_df = crime_sample_df.drop(columns=['Date'])
crime_sample_df = crime_sample_df.astype(int)

In [18]:
X = crime_sample_df.drop('Arrest', axis=1)
y = crime_sample_df['Arrest']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4, random_state=7)

In [19]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

In [20]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

y_pred = knn.predict(X_valid_scaled)

accuracy = accuracy_score(y_valid, y_pred)

print(accuracy)

0.8683733649632631


In [21]:
result = []

for k in range(1, 13, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    accuracy = accuracy_score(y_valid, knn.predict(X_valid_scaled))

    result.append({'k': k, 'accuracy': accuracy})

results_df = pd.DataFrame(result)
results_df

,k,accuracy
0,1,0.823202
1,3,0.856875
2,5,0.868373
3,7,0.872228
4,9,0.874238
5,11,0.875556


### Logistic Regression

In [23]:
logit_reg = LogisticRegression(penalty="l2", C=1e42, solver='liblinear')
logit_reg.fit(X_train, y_train)

LogisticRegression(C=1e+42, solver='liblinear')

In [24]:
y_pred = logit_reg.predict(X_valid)
print('Accuracy:', accuracy_score(y_valid, y_pred))
print('Intercept ', logit_reg.intercept_[0])

Accuracy: 0.8739744983690817
Intercept  -0.2042571907778676


In [25]:
logit_proba = logit_reg.predict_proba(X_valid)
logit_result = pd.DataFrame({
    'actual': y_valid.values,
    'p(0)': logit_proba[:, 0],
    'p(1)': logit_proba[:, 1],
    'predicted': logit_reg.predict(X_valid),})
logit_result = logit_result.sort_values(by='p(1)', ascending=False)

df = logit_result.sort_values(by='p(1)', ascending=False)

fig, ax = plt.subplots(figsize=(7, 5))
ax = gainsChart(df.actual, label='Logistic Regression', color='C1', ax=ax)
ax.legend()
plt.title('Gains Chart')
plt.show()

/var/folders/ws/skn1jvl12xq48r7t01qb74pm0000gn/T/ipykernel_42495/2785638532.py:15: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Compare the performance of knn and logit_regression

In [27]:
print("Logistic Regression")
classificationSummary(y_valid, logit_reg.predict(X_valid), class_names=['No Arrest', 'Arrest'])
print()

print("KNN (k=11)")
classificationSummary(y_valid, knn.predict(X_valid_scaled), class_names=['No Arrest', 'Arrest'])

Logistic Regression
Confusion Matrix (Accuracy 0.8740)

          Prediction
   Actual No Arrest    Arrest
No Arrest     22168       477
   Arrest      3348      4358

KNN (k=11)
Confusion Matrix (Accuracy 0.8756)

          Prediction
   Actual No Arrest    Arrest
No Arrest     21823       822
   Arrest      2955      4751


In [28]:
df = logit_result.sort_values(by='p(1)', ascending=False)
knn_proba = knn.predict_proba(X_valid)
knn_result = pd.DataFrame({
    'actual': y_valid.values,
    'p(0)': knn_proba[:, 0],
    'p(1)': knn_proba[:, 1],
    'predicted': knn.predict(X_valid),})
knn_result = knn_result.sort_values(by='p(1)', ascending=False)

fig, ax = plt.subplots(figsize=(7, 5))
ax = gainsChart(df.actual, label='Logistic Regression', color='C1', ax=ax)
ax = gainsChart(knn_result.actual, label='KNN (k=11)', color='C0', ax=ax)

ax.legend()
plt.title('Gains Chart Comparison')
plt.tight_layout()
plt.show()

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
/var/folders/ws/skn1jvl12xq48r7t01qb74pm0000gn/T/ipykernel_42495/2472819639.py:17: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Decision Tree

In [30]:
fullClassTree = tree.DecisionTreeClassifier(random_state=0)
fullClassTree.fit(X_train, y_train)

text_representation = tree.export_text(fullClassTree, feature_names=list(X_train.columns))

fig = plt.figure(figsize=(25, 20))
_ = tree.plot_tree(fullClassTree, feature_names=X_train.columns, filled=True)
fig.savefig("decision_tree_crime_full.png")

In [31]:
treeClassifier = tree.DecisionTreeClassifier(random_state=0)
scores = cross_val_score(treeClassifier, X, y, cv=5)
print(f'Overall Accuracy: {scores.mean():.3f} (+/- {scores.std() * 2:.3f})')

Overall Accuracy: 0.817 (+/- 0.005)


In [32]:
smallClassTree = tree.DecisionTreeClassifier(
    max_depth=5,
    min_samples_split=20,
    min_impurity_decrease=0.01,
    random_state=0
)
smallClassTree.fit(X_train, y_train)

text_representation = tree.export_text(smallClassTree, feature_names=list(X_train.columns))
#print(text_representation)

fig = plt.figure(figsize=(25, 20))
_ = tree.plot_tree(smallClassTree, feature_names=X_train.columns, filled=True)
fig.savefig("decision_tree_crime_small.png")

In [33]:
y_pred_small = smallClassTree.predict(X_valid)
accuracy = accuracy_score(y_valid, y_pred_small)
print(f'Smaller Tree Overall Accuracy: {scores.mean():.3f} (+/- {scores.std() * 2:.3f})')

Smaller Tree Overall Accuracy: 0.817 (+/- 0.005)


In [34]:
print(fullClassTree.get_params())
print(smallClassTree.get_params())
print(f'Full tree depth: {fullClassTree.get_depth()}')
print(f'Small tree depth: {smallClassTree.get_depth()}')

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': 0, 'splitter': 'best'}
{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.01, 'min_samples_leaf': 1, 'min_samples_split': 20, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': 0, 'splitter': 'best'}
Full tree depth: 100
Small tree depth: 4


### Compare Tree & KNN & LReg

In [36]:
print("Logistic Regression")
classificationSummary(y_valid, logit_reg.predict(X_valid), class_names=['No Arrest', 'Arrest'])
print()

print("KNN (k=11)")
classificationSummary(y_valid, knn.predict(X_valid_scaled), class_names=['No Arrest', 'Arrest'])
print()

print("Decision Tree (max_depth=4)")
classificationSummary(y_valid, smallClassTree.predict(X_valid), class_names=['No Arrest', 'Arrest'])

Logistic Regression
Confusion Matrix (Accuracy 0.8740)

          Prediction
   Actual No Arrest    Arrest
No Arrest     22168       477
   Arrest      3348      4358

KNN (k=11)
Confusion Matrix (Accuracy 0.8756)

          Prediction
   Actual No Arrest    Arrest
No Arrest     21823       822
   Arrest      2955      4751

Decision Tree (max_depth=4)
Confusion Matrix (Accuracy 0.8600)

          Prediction
   Actual No Arrest    Arrest
No Arrest     22225       420
   Arrest      3828      3878


In [37]:
df = logit_result.sort_values(by='p(1)', ascending=False)

knn_proba = knn.predict_proba(X_valid)
knn_result = pd.DataFrame({
    'actual': y_valid.values,
    'p(0)': knn_proba[:, 0],
    'p(1)': knn_proba[:, 1],
    'predicted': knn.predict(X_valid),
}).sort_values(by='p(1)', ascending=False)

tree_proba = smallClassTree.predict_proba(X_valid)
tree_result = pd.DataFrame({
    'actual': y_valid.values,
    'p(1)': tree_proba[:, 1]
}).sort_values(by='p(1)', ascending=False)

fig, ax = plt.subplots(figsize=(7, 5))

ax = gainsChart(df.actual, label='Logistic Regression', color='C1', ax=ax)
ax = gainsChart(knn_result.actual, label='KNN (k=11)', color='C0', ax=ax)
ax = gainsChart(tree_result.actual, label='Decision Tree (max_depth=5)', color='C2', ax=ax)

ax.legend()
plt.title('Gains Chart Comparison')
plt.tight_layout()
plt.show()

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
/var/folders/ws/skn1jvl12xq48r7t01qb74pm0000gn/T/ipykernel_42495/510202289.py:26: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
